In [14]:
import pandas as pd
from PIL import Image
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import torchvision.models as models


# data preparation:do not need to run the below codes 

In [23]:
# Load the dataset
data = pd.read_csv('Bookshelves_5_labeled_df.csv', encoding='ISO-8859-1')


In [21]:
data

,txt,confidence,vertices,boundBox,slopes,font,word_len,direction,center_point,crop_idx,Left,Top,Label
0,Don't,100,"[(1039, 566), (1066, 563), (1067, 576), (1040,...","{'Width': 28, 'Height': 16, 'Left': 1039, 'Top...","(0.111, -13.0, 0.111, -13.0)",13.038405,27.166155,horizontal,"(1053.0, 571.0)","(0, 0)",1039,563,0
1,Lik,100,"[(1066, 563), (1080, 561), (1081, 574), (1067,...","{'Width': 15, 'Height': 15, 'Left': 1066, 'Top...","(0.143, -13.0, 0.143, -13.0)",13.038405,14.142136,horizontal,"(1073.5, 568.5)","(0, 0)",1066,561,0
2,AND,100,"[(1112, 307), (1172, 309), (1170, 364), (1110,...","{'Width': 62, 'Height': 57, 'Left': 1110, 'Top...","(-0.033, 27.5, -0.033, 27.5)",55.036352,60.033324,horizontal,"(1141.0, 335.5)","(0, 0)",1110,307,0
3,FALL,100,"[(1119, 346), (1187, 353), (1185, 373), (1117,...","{'Width': 70, 'Height': 27, 'Left': 1117, 'Top...","(-0.103, 10.0, -0.088, 10.5)",20.597387,68.311769,horizontal,"(1152.0, 359.75)","(0, 0)",1117,346,0
4,LOVE,100,"[(1119, 502), (1183, 508), (1181, 530), (1117,...","{'Width': 66, 'Height': 28, 'Left': 1117, 'Top...","(-0.094, 11.0, -0.094, 11.0)",22.090722,64.280635,horizontal,"(1150.0, 516.0)","(0, 0)",1117,502,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,the,100,"[(1039, 286), (1034, 324), (1007, 320), (1012,...","{'Width': 32, 'Height': 41, 'Left': 1007, 'Top...","(7.6, -0.148, 7.4, -0.111)",27.230422,37.831923,vertical,"(1023.0, 303.25)","(2, 1)",1007,283,1
651,from,100,"[(1048, 219), (1040, 281), (1013, 277), (1021,...","{'Width': 35, 'Height': 65, 'Left': 1013, 'Top...","(7.75, -0.148, 7.625, -0.111)",27.230422,62.018176,vertical,"(1030.5, 248.25)","(2, 1)",1013,216,1
652,Half,100,"[(1056, 159), (1049, 218), (1021, 214), (1028,...","{'Width': 35, 'Height': 63, 'Left': 1021, 'Top...","(8.429, -0.143, 8.429, -0.143)",28.284271,59.413803,vertical,"(1038.5, 186.5)","(2, 1)",1021,155,1
653,One,100,"[(1063, 103), (1056, 155), (1029, 151), (1035,...","{'Width': 34, 'Height': 56, 'Left': 1029, 'Top...","(7.429, -0.148, 8.667, -0.143)",27.789480,52.407024,vertical,"(1045.75, 127.0)","(2, 1)",1029,99,1


In [10]:
def crop_image_from_vertices(image_path, vertices, expand=10):
    """
    Crop image based on vertices with expansion.
    Arguments:
    - image_path: str, path to the image file
    - vertices: list of tuples, each tuple is a coordinate (x, y)
    - expand: int, number of pixels to expand the crop area
    """
    img = Image.open(image_path)
    min_x = min([v[0] for v in vertices])
    min_y = min([v[1] for v in vertices])
    max_x = max([v[0] for v in vertices])
    max_y = max([v[1] for v in vertices])
    
    # Expand the box by 'expand' pixels
    min_x = max(0, min_x - expand)
    min_y = max(0, min_y - expand)
    max_x = min(img.width, max_x + expand)
    max_y = min(img.height, max_y + expand)
    
    cropped_image = img.crop((min_x, min_y, max_x, max_y))
    return cropped_image


In [22]:
# Define image paths based on crop_idx (you should adjust these paths according to your file structure)
image_paths = {
    '(0, 0)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_5.jpg_0_0.jpg',
    '(0, 1)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_5.jpg_0_1.jpg',
    '(1, 0)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_5.jpg_1_0.jpg',
    '(1, 1)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_5.jpg_1_1.jpg',
    '(2, 0)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_5.jpg_2_0.jpg',
    '(2, 1)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_5.jpg_2_1.jpg',
    # Add more paths as per the crop_idx values
}

# Iterate over the rows of the dataset
for index, row in data.iterrows():
    vertices = eval(row['vertices'])  # Assuming vertices are stored as a string representation of list of tuples
    crop_idx = row['crop_idx']
    image_path = image_paths[crop_idx]  # Get the path based on crop_idx
    
    # Crop the image
    cropped_image = crop_image_from_vertices(image_path, vertices, expand=20)  # You can adjust the expansion here
    
    # Save or process the cropped image
    cropped_image.save(f'cropped_images/cropped_{index}.jpg')  # Save each cropped image


In [25]:
data = pd.read_csv('Bookshelves_1_df.csv', encoding='ISO-8859-1')
data

,txt,confidence,vertices,boundBox,slopes,font,word_len,direction,center_point,crop_idx,Left,Top,Label
0,THE,100,"[(83, 360), (85, 402), (69, 403), (67, 361)]","{'Width': 18, 'Height': 43, 'Left': 67, 'Top':...","(-21.0, 0.062, -21.0, 0.062)",16.031220,42.047592,vertical,"(76.0, 381.5)","(0, 0)",67,360,1
1,TIPPING,100,"[(85, 412), (88, 501), (72, 502), (69, 413)]","{'Width': 19, 'Height': 90, 'Left': 69, 'Top':...","(-29.667, 0.062, -29.667, 0.062)",16.031220,89.050547,vertical,"(78.5, 457.0)","(0, 0)",69,412,1
2,POINT,100,"[(89, 511), (92, 578), (76, 579), (73, 512)]","{'Width': 19, 'Height': 68, 'Left': 73, 'Top':...","(-22.333, 0.062, -22.333, 0.062)",16.031220,67.067131,vertical,"(82.5, 545.0)","(0, 0)",73,511,1
3,MALCOLM,100,"[(90, 621), (95, 722), (80, 723), (75, 622)]","{'Width': 20, 'Height': 102, 'Left': 75, 'Top'...","(-20.2, 0.067, -20.2, 0.067)",15.033296,101.123687,vertical,"(85.0, 672.0)","(0, 0)",75,621,1
4,GLADWELL,100,"[(96, 735), (102, 846), (87, 847), (81, 736)]","{'Width': 21, 'Height': 112, 'Left': 81, 'Top'...","(-18.5, 0.067, -18.5, 0.067)",15.033296,111.162044,vertical,"(91.5, 791.0)","(0, 0)",81,735,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,ERIC,100,"[(1065, 152), (1104, 152), (1104, 164), (1065,...","{'Width': 39, 'Height': 12, 'Left': 1065, 'Top...","(-0.0, 1000.0, 0.0, 1000.0)",12.000000,39.000000,horizontal,"(1084.5, 158.0)","(1, 1)",1065,152,1
541,DUDEN,100,"[(1173, 194), (1272, 194), (1272, 218), (1173,...","{'Width': 99, 'Height': 24, 'Left': 1173, 'Top...","(-0.0, 1000.0, 0.0, 1000.0)",24.000000,99.000000,horizontal,"(1222.5, 206.0)","(1, 1)",1173,194,1
542,§¢§Ñ§Ý§Ñ§Ý§â§è§Ñ§â§Ú,100,"[(1237, 320), (1219, 513), (1183, 510), (1201,...","{'Width': 54, 'Height': 196, 'Left': 1183, 'To...","(10.722, -0.083, 10.722, -0.083)",36.124784,193.837561,vertical,"(1210.0, 415.0)","(1, 1)",1183,317,0
543,+,100,"[(1209, 596), (1207, 621), (1184, 619), (1186,...","{'Width': 25, 'Height': 27, 'Left': 1184, 'Top...","(12.5, -0.087, 12.5, -0.087)",23.086793,25.079872,vertical,"(1196.5, 607.5)","(1, 1)",1184,594,0


In [26]:
# Define image paths based on crop_idx (you should adjust these paths according to your file structure)
image_paths = {
    '(0, 0)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_1.jpg_0_0.jpg',
    '(0, 1)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_1.jpg_0_1.jpg',
    '(1, 0)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_1.jpg_1_0.jpg',
    '(1, 1)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_1.jpg_1_1.jpg',
#     '(2, 0)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_5.jpg_2_0.jpg',
#     '(2, 1)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_5.jpg_2_1.jpg',
#     # Add more paths as per the crop_idx values
}

# Iterate over the rows of the dataset
for index, row in data.iterrows():
    vertices = eval(row['vertices'])  # Assuming vertices are stored as a string representation of list of tuples
    crop_idx = row['crop_idx']
    image_path = image_paths[crop_idx]  # Get the path based on crop_idx
    
    # Crop the image
    cropped_image = crop_image_from_vertices(image_path, vertices, expand=20)  # You can adjust the expansion here
    
    # Save or process the cropped image
    cropped_image.save(f'cropped_images_1/cropped_{index}.jpg')  # Save each cropped image


In [27]:
data = pd.read_csv('Bookshelves_4_df.csv', encoding='ISO-8859-1')
data

,txt,confidence,vertices,boundBox,slopes,font,word_len,direction,center_point,crop_idx,Left,Top,Label
0,med,100,"[(101, 75), (102, 99), (89, 99), (88, 75)]","{'Width': 14, 'Height': 24, 'Left': 88, 'Top':...","(-24.0, 0.0, -24.0, -0.0)",13.000000,24.020824,vertical,"(95.0, 87.0)","(0, 0)",88,75,0
1,all,100,"[(102, 106), (102, 124), (88, 124), (88, 106)]","{'Width': 14, 'Height': 18, 'Left': 88, 'Top':...","(1000.0, 0.0, 1000.0, -0.0)",14.000000,18.000000,vertical,"(95.0, 115.0)","(0, 0)",88,106,1
2,the,100,"[(102, 126), (102, 146), (89, 146), (89, 126)]","{'Width': 13, 'Height': 20, 'Left': 89, 'Top':...","(1000.0, 0.0, 1000.0, -0.0)",13.000000,20.000000,vertical,"(95.5, 136.0)","(0, 0)",89,126,1
3,bright,100,"[(103, 152), (104, 191), (90, 191), (89, 152)]","{'Width': 15, 'Height': 39, 'Left': 89, 'Top':...","(-39.0, 0.0, -39.0, -0.0)",14.000000,39.012818,vertical,"(96.5, 171.5)","(0, 0)",89,152,1
4,?,100,"[(110, 353), (111, 380), (90, 381), (89, 354)]","{'Width': 22, 'Height': 28, 'Left': 89, 'Top':...","(-27.0, 0.048, -27.0, 0.048)",21.023796,27.018512,vertical,"(100.0, 367.0)","(0, 0)",89,353,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,PARTY,100,"[(420, 259), (473, 254), (474, 267), (421, 272)]","{'Width': 54, 'Height': 18, 'Left': 420, 'Top'...","(0.094, -13.0, 0.094, -13.0)",13.038405,53.235327,horizontal,"(447.0, 263.0)","(4, 1)",420,254,1
447,UGLYDOLL,100,"[(429, 75), (544, 59), (547, 81), (432, 98)]","{'Width': 118, 'Height': 39, 'Left': 429, 'Top...","(0.139, -7.333, 0.148, -7.667)",22.699215,116.178720,horizontal,"(488.0, 78.25)","(4, 1)",429,59,1
448,GOODIES,100,"[(441, 185), (523, 169), (526, 187), (444, 203)]","{'Width': 85, 'Height': 34, 'Left': 441, 'Top'...","(0.195, -6.0, 0.195, -6.0)",18.248288,83.546394,horizontal,"(483.5, 186.0)","(4, 1)",441,169,1
449,IN,100,"[(453, 160), (496, 151), (500, 171), (457, 181)]","{'Width': 47, 'Height': 30, 'Left': 453, 'Top'...","(0.209, -5.0, 0.233, -5.25)",20.886818,44.039623,horizontal,"(476.5, 165.75)","(4, 1)",453,151,0


In [30]:
# Define image paths based on crop_idx (you should adjust these paths according to your file structure)
image_paths = {
    '(0, 0)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_4.jpg_0_0.jpg',
    '(0, 1)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_4.jpg_0_1.jpg',
    '(1, 0)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_4.jpg_1_0.jpg',
    '(1, 1)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_4.jpg_1_1.jpg',
    '(2, 0)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_4.jpg_2_0.jpg',
    '(2, 1)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_4.jpg_2_1.jpg',
    '(3, 0)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_4.jpg_3_0.jpg',
    '(3, 1)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_4.jpg_3_1.jpg',
    '(4, 0)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_4.jpg_4_0.jpg',
    '(4, 1)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_4.jpg_4_1.jpg',
    # Add more paths as per the crop_idx values
}

# Iterate over the rows of the dataset
for index, row in data.iterrows():
    vertices = eval(row['vertices'])  # Assuming vertices are stored as a string representation of list of tuples
    crop_idx = row['crop_idx']
    image_path = image_paths[crop_idx]  # Get the path based on crop_idx
    
    # Crop the image
    cropped_image = crop_image_from_vertices(image_path, vertices, expand=20)  # You can adjust the expansion here
    
    # Save or process the cropped image
    cropped_image.save(f'cropped_images_4/cropped_{index}.jpg')  # Save each cropped image


In [32]:
data = pd.read_csv('IMG_7940_labelled_df.csv', encoding='ISO-8859-1')
data

,txt,confidence,vertices,boundBox,slopes,font,word_len,direction,center_point,crop_idx,Left,Top,Label
0,ated,100,"[(0, 189), (72, 195), (69, 228), (0, 222)]","{'Width': 72, 'Height': 39, 'Left': 0, 'Top': ...","(-0.083, 11.0, -0.087, 1000.0)",33.068042,70.754973,horizontal,"(35.25, 208.5)","(0, 0)",0,189,0
1,808.3,100,"[(184, 1168), (241, 1170), (241, 1185), (184, ...","{'Width': 57, 'Height': 17, 'Left': 184, 'Top'...","(-0.035, 1000.0, -0.035, 1000.0)",15.000000,57.035077,horizontal,"(212.5, 1176.5)","(0, 0)",184,1168,0
2,BUC,100,"[(184, 1193), (218, 1194), (217, 1208), (184, ...","{'Width': 34, 'Height': 15, 'Left': 184, 'Top'...","(-0.029, 14.0, -0.03, 1000.0)",14.017834,33.514925,horizontal,"(200.75, 1200.5)","(0, 0)",184,1193,0
3,A,100,"[(220, 347), (220, 383), (186, 383), (186, 347)]","{'Width': 34, 'Height': 36, 'Left': 186, 'Top'...","(1000.0, 0.0, 1000.0, -0.0)",34.000000,36.000000,vertical,"(203.0, 365.0)","(0, 0)",186,347,1
4,Writer's,100,"[(220, 397), (222, 557), (188, 557), (186, 397)]","{'Width': 36, 'Height': 160, 'Left': 186, 'Top...","(-80.0, 0.0, -80.0, -0.0)",34.000000,160.012499,vertical,"(204.0, 477.0)","(0, 0)",186,397,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,POETRY,100,"[(3870, 560), (3858, 639), (3839, 636), (3851,...","{'Width': 31, 'Height': 82, 'Left': 3839, 'Top...","(6.583, -0.158, 6.583, -0.158)",19.235384,79.906195,vertical,"(3854.5, 598.0)","(0, 0)",3839,557,1
420,OF,100,"[(3877, 517), (3873, 544), (3853, 541), (3857,...","{'Width': 24, 'Height': 30, 'Left': 3853, 'Top...","(6.75, -0.15, 6.75, -0.15)",20.223748,27.294688,vertical,"(3865.0, 529.0)","(0, 0)",3853,514,1
421,BOOK,100,"[(3887, 451), (3879, 504), (3860, 502), (3868,...","{'Width': 27, 'Height': 56, 'Left': 3860, 'Top...","(6.625, -0.105, 6.75, -0.158)",19.170179,54.094875,vertical,"(3873.5, 476.25)","(0, 0)",3860,448,1
422,THE,100,"[(3896, 395), (3890, 436), (3870, 433), (3876,...","{'Width': 26, 'Height': 44, 'Left': 3870, 'Top...","(6.833, -0.15, 6.833, -0.15)",20.223748,41.436699,vertical,"(3883.0, 414.0)","(0, 0)",3870,392,1


In [33]:
# Define image paths based on crop_idx (you should adjust these paths according to your file structure)
image_paths = {
    '(0, 0)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/IMG_7940.jpeg_0_0.jpg',
#     '(0, 1)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_5.jpg_0_1.jpg',
#     '(1, 0)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_5.jpg_1_0.jpg',
#     '(1, 1)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_5.jpg_1_1.jpg',
#     '(2, 0)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_5.jpg_2_0.jpg',
#     '(2, 1)': 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google/Bookshelves_5.jpg_2_1.jpg',
#     # Add more paths as per the crop_idx values
}

# Iterate over the rows of the dataset
for index, row in data.iterrows():
    vertices = eval(row['vertices'])  # Assuming vertices are stored as a string representation of list of tuples
    crop_idx = row['crop_idx']
    image_path = image_paths[crop_idx]  # Get the path based on crop_idx
    
    # Crop the image
    cropped_image = crop_image_from_vertices(image_path, vertices, expand=20)  # You can adjust the expansion here
    
    # Save or process the cropped image
    cropped_image.save(f'cropped_images_79/cropped_{index}.jpg')  # Save each cropped image


In [37]:
# CSV文件路径列表
csv_files = [
    'Bookshelves_1_df.csv',
    'Bookshelves_4_df.csv',  # 请确保路径和文件名是正确的
    'Bookshelves_5_labeled_df.csv',
    'IMG_7940_labelled_df.csv'  # 请确保路径和文件名是正确的
]

# 初始化空的DataFrame列表，用于存放各个文件的label列
label_dfs = []

# 遍历文件路径列表，加载每个文件的label列
for csv_file in csv_files:
    df = pd.read_csv(csv_file, encoding='ISO-8859-1')
    # 取label列，并添加到列表中
    # 确保CSV文件中含有'Label'这一列
    label_dfs.append(df['Label'])

# 使用concat合并所有的DataFrame
final_labels = pd.concat(label_dfs, ignore_index=True)


In [51]:
import re

def natural_sort_key(s):
    # 使用正则表达式捕获文件名中的所有数字，并转换为整数
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]


In [52]:
# 图像目录和CSV文件映射
data_info = {
    "cropped_images_1": "Bookshelves_1_df.csv",
    "cropped_images_4": "Bookshelves_4_df.csv",
    "cropped_images_5": "Bookshelves_5_labeled_df.csv",
    "cropped_images_79": "IMG_7940_labelled_df.csv"
}

base_path = 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves'  # 请调整为实际路径

# 初始化空DataFrame用于存储所有数据
all_data = pd.DataFrame()

# 处理每个文件夹和CSV文件
for folder, csv_file in data_info.items():
    # 构建完整的CSV文件路径
    csv_path = os.path.join(base_path, csv_file)
    # 读取CSV文件
    df = pd.read_csv(csv_path,encoding='ISO-8859-1')

    # 获取图像文件夹路径
    folder_path = os.path.join(base_path, folder)
    # 获取该文件夹内所有图像的路径
    img_files = sorted(
        [os.path.join(folder, file) for file in os.listdir(folder_path) if file.endswith(('png', 'jpg', 'jpeg'))],
        key=natural_sort_key  # 使用自定义的排序函数
    )
    # 检查图像数量和CSV文件行数是否匹配
    if len(img_files) != len(df):
        print(f"Warning: Number of images in {folder} does not match number of labels in {csv_file}")

    # 添加图像路径到DataFrame
    df['image_path'] = img_files

    # 将当前DataFrame添加到总的DataFrame
    all_data = pd.concat([all_data, df], ignore_index=True)


In [54]:
# 查看DataFrame的前几行
print(all_data.head())

# 查看数据的总行数
print(f"Total number of entries: {len(all_data)}")

print(all_data[['image_path']].head(20))  # 输出前20个条目来检查排序



        txt  confidence                                       vertices  \
0       THE         100   [(83, 360), (85, 402), (69, 403), (67, 361)]   
1   TIPPING         100   [(85, 412), (88, 501), (72, 502), (69, 413)]   
2     POINT         100   [(89, 511), (92, 578), (76, 579), (73, 512)]   
3   MALCOLM         100   [(90, 621), (95, 722), (80, 723), (75, 622)]   
4  GLADWELL         100  [(96, 735), (102, 846), (87, 847), (81, 736)]   

                                            boundBox  \
0  {'Width': 18, 'Height': 43, 'Left': 67, 'Top':...   
1  {'Width': 19, 'Height': 90, 'Left': 69, 'Top':...   
2  {'Width': 19, 'Height': 68, 'Left': 73, 'Top':...   
3  {'Width': 20, 'Height': 102, 'Left': 75, 'Top'...   
4  {'Width': 21, 'Height': 112, 'Left': 81, 'Top'...   

                             slopes       font    word_len direction  \
0      (-21.0, 0.062, -21.0, 0.062)  16.031220   42.047592  vertical   
1  (-29.667, 0.062, -29.667, 0.062)  16.031220   89.050547  vertical   
2 

In [55]:
# 指定要保存的CSV文件路径
csv_output_path = 'all_data.csv'  # 替换为实际的路径

# 保存DataFrame到CSV
all_data.to_csv(csv_output_path, index=False)


In [7]:
import torch
print(torch.__version__)


1.12.1


# Start running codes from here

In [5]:
import torch
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# 加载预训练的ResNet-50模型
model = models.resnet50(pretrained=True)

# 冻结所有层，防止在训练中更新权重
for param in model.parameters():
    param.requires_grad = False

# 替换最后的全连接层以适应你的分类任务，假设是二分类
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)


C:\Users\19945\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\19945\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\19945/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:01<00:00, 76.6MB/s]


In [11]:
class BookshelfDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, -1]  # Assuming image path is last column
        image = Image.open(img_path).convert('RGB')
        label = int(self.dataframe.iloc[idx, self.dataframe.columns.get_loc('Label')])

        if self.transform:
            image = self.transform(image)

        return image, label

# Transforms for the input data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

all_data = pd.read_csv('all_data.csv')


# Create the Dataset and DataLoader
dataset = BookshelfDataset(dataframe=all_data, transform=transform)
loader = DataLoader(dataset, batch_size=64, shuffle=True)


In [15]:
# 假设 all_data 是你已经加载的完整DataFrame
all_data = pd.read_csv('all_data.csv')  # 加载数据，如果已经加载可以省略这步

# 划分数据集为训练集和验证集，这里使用80:20的比例
train_df, valid_df = train_test_split(all_data, test_size=0.2, random_state=42)  # random_state确保结果可复现

# 可以查看一下分离后的数据集大小
print(f"Training data size: {train_df.shape}")
print(f"Validation data size: {valid_df.shape}")


Training data size: (1660, 14)
Validation data size: (415, 14)


In [16]:
# 使用上述定义的BookshelfDataset类来创建训练集和验证集的Dataset
train_dataset = BookshelfDataset(train_df, transform=transform)
valid_dataset = BookshelfDataset(valid_df, transform=transform)

# 创建DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)  # 只优化新的全连接层

# # 训练函数
# def train_model(model, loader, num_epochs=10):
#     model.train()
#     for epoch in range(num_epochs):
#         for images, labels in loader:
#             images, labels = images.to(device), labels.to(device)
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#         print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# train_model(model, loader)


Epoch 1/10, Loss: 0.2642759680747986
Epoch 2/10, Loss: 0.5055345892906189
Epoch 3/10, Loss: 0.30122941732406616
Epoch 4/10, Loss: 0.23912951350212097
Epoch 5/10, Loss: 0.4211134612560272
Epoch 6/10, Loss: 0.22446011006832123
Epoch 7/10, Loss: 0.19313503801822662
Epoch 8/10, Loss: 0.15583078563213348
Epoch 9/10, Loss: 0.3499011993408203
Epoch 10/10, Loss: 0.14274989068508148


In [10]:
# train_model(model, loader)


Epoch 1/10, Loss: 0.5167664885520935
Epoch 2/10, Loss: 0.21976123750209808
Epoch 3/10, Loss: 0.16238144040107727
Epoch 4/10, Loss: 0.2929905354976654
Epoch 5/10, Loss: 0.47995322942733765
Epoch 6/10, Loss: 0.25192514061927795
Epoch 7/10, Loss: 0.28143516182899475
Epoch 8/10, Loss: 0.422868013381958
Epoch 9/10, Loss: 0.4227026104927063
Epoch 10/10, Loss: 0.109414242208004


In [12]:
def evaluate_model(model, loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy


In [17]:
def train_model(model, train_loader, valid_loader, num_epochs=50):
    best_accuracy = 0
    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            accuracy = evaluate_model(model, valid_loader)
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}, Accuracy: {accuracy}%')
            
            # Save the model if the accuracy is the best
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                torch.save(model.state_dict(), f'model_epoch_{epoch+1}.pth')
                print(f"Best model saved with accuracy: {accuracy}%")


In [ ]:
# 使用之前定义的BookshelfDataset和transform
train_dataset = BookshelfDataset(train_df, transform=transform)  # train_df需要被定义
valid_dataset = BookshelfDataset(valid_df, transform=transform)  # valid_df需要被定义

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)


In [18]:
train_model(model, train_loader, valid_loader, num_epochs=100)


Epoch [10/100], Loss: 0.46218565106391907, Accuracy: 90.8433734939759%
Best model saved with accuracy: 90.8433734939759%


KeyboardInterrupt: 